
Задание

Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

Описание данных 

- date – дата совершения события
- event - событие

app_install – установка приложения
app_start – открыть приложения
registration – зарегистрироваться 
search – перейти на страницу поиска товаров (каталог)
open_item – открыть товар
choose_item – отправить товар в корзину
tap_basket – перейти в корзину
purchase – подтверждение покупки
- gender – пол пользователя
- os_name – платформа пользователя
- city – город пользователя
- device_id – идентификатор устройства пользователя
- urm_source – канал, с которого пришел пользователь

yandex-direct – Яндекс директ
google_ads – реклама в Google
vk_ads – реклама в ВК
instagram_ads – реклама в instagram
facebook_ads – реклама в facebook
referral – акция «приведи друга»
Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  
- purchase_sum – стоимость покупки (при совершении события ‘purchase’)

Обратите внимание на следующее:

- В выгрузке только уникальные действия пользователей за каждый день 
- Можно миновать стадию установки приложения, если оно было установлено ранее
- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline 

In [2]:
df = pd.read_csv('KC_case_data .csv')

In [3]:
df.date = pd.to_datetime(df.date, dayfirst=True)

In [4]:
df['year_month'] = pd.to_datetime(df.date).dt.to_period('M')

In [5]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,year_month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01


In [6]:
df['event'].value_counts()

app_start      748705
search         708639
choose_item    538669
tap_basket     377665
app_install    154597
purchase       141383
register        78310
Name: event, dtype: int64

In [7]:
installs = df[df['event'] == 'app_install'][['date', 'device_id']]

In [8]:
installs == installs.drop_duplicates('device_id')

,date,device_id
2515061,True,True
2515062,True,True
2515063,True,True
2515064,True,True
2515065,True,True
...,...,...
2669653,True,True
2669654,True,True
2669655,True,True
2669656,True,True


In [9]:
installs = installs.rename(columns = {'date': 'install_date'})

In [10]:
#добавляем дату установки каждого пользователя в основной датафрейм
df = df.merge(installs, how = 'left', on = 'device_id')

In [11]:
df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,year_month,install_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01,2020-01-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01,NaT
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01,2020-01-01
...,...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03,2020-03-28
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03,2020-03-31
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03,2020-03-31
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03,2020-03-06


In [39]:
#считаем первую дату покупки
purchases = df[df['event'] == 'purchase'][['date', 'device_id']]
purchases = purchases.rename(columns = {'date': 'first_purchase_date'})

In [13]:
purchases = purchases.drop_duplicates('device_id')

In [14]:
df = df.merge(purchases, how = 'left', on = 'device_id')

In [15]:
df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,year_month,install_date,first_purchase_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01,2020-01-01,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01,2020-01-01,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01,NaT,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01,2020-01-01,NaT
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01,2020-01-01,NaT
...,...,...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03,2020-03-28,2020-03-31
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03,2020-03-31,2020-03-31
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03,2020-03-31,2020-03-31
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03,2020-03-06,2020-03-31


In [15]:
df.to_csv('df.csv', index=False)

Retention в Табло
https://public.tableau.com/app/profile/victoriia5838/viz/BookRet/Dashboard1

Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

https://public.tableau.com/app/profile/victoriia5838/viz/BookRet/Dashboard1

Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

В ответ впишите один из шагов, конверсия из которого (в следующий шаг) самая низкая, в таком формате: «Поиск», «Добавление товара», «Переход в корзину», «Регистрация».

In [16]:
first_registration = df[df['event'] == 'register'].sort_values('date').drop_duplicates('device_id')

In [17]:
df['first_registration'] = df['device_id'].map(first_registration.set_index('device_id')['date'])

In [18]:
df['first_registration'].head()

0   2020-01-07
1   2020-01-01
2   2020-01-01
3   2020-01-01
4          NaT
Name: first_registration, dtype: datetime64[ns]

In [19]:
alredy_reg = df[df['first_registration'] < df['date']]

In [20]:
alredy_reg.groupby('event')['device_id'].nunique()

event
app_start      40991
choose_item    37926
purchase       24880
search         40482
tap_basket     34517
Name: device_id, dtype: int64

In [34]:
print('Конверсия в поиск из открытия', round((40482/40991)* 100, 2))

Конверсия в поиск из открытия 98.76


In [35]:
print('Конверсия в добавление товара из поиска', round((37926/40482)* 100, 2))

Конверсия в добавление товара из поиска 93.69


In [36]:
print('Конверсия из добавления в переход в корзину', round((34517/37926)* 100, 2))

Конверсия из добавления в переход в корзину 91.01


In [37]:
print('Конверсия из перехода в корзину в покупку', round((24880/34517)* 100, 2))

Конверсия из перехода в корзину в покупку 72.08


Данные по затратам на рекламу:

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

Какой платный канал привлечения имеет самый высокий ROMI? 

In [25]:
df.groupby('utm_source')['purchase_sum'].sum()

utm_source
-                21449749.5
facebook_ads     12249901.0
google_ads       12868276.0
instagram_ads    14546969.0
referal           8837044.5
vk_ads           16389652.5
yandex-direct    13915368.0
Name: purchase_sum, dtype: float64

In [26]:
print('ROMI Facebook', round((12249901 / 8590498 - 1) *100, 2), "%")

ROMI Facebook 42.6 %


In [27]:
print('ROMI Google', round((12868276 / 10534878 - 1) *100, 2), "%")

ROMI Google 22.15 %


In [28]:
print('ROMI Yandex', round((13915368 / 10491707 - 1) *100, 2), "%")

ROMI Yandex 32.63 %


In [29]:
print('ROMI Instagram', round((14546969 / 8561626 - 1) *100, 2), "%")

ROMI Instagram 69.91 %


In [30]:
print('ROMI VK', round((16389652.5 / 9553531 - 1) *100, 2), "%")

ROMI VK 71.56 %


Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

In [40]:
purchases = df[df['event'] == 'purchase'][['date', 'device_id', 'purchase_sum', 'utm_source']]
purchases = purchases.sort_values('date').drop_duplicates('device_id')

In [42]:
purchases.groupby('utm_source').median()

,device_id,purchase_sum
utm_source,,
-,10972119.0,398.5
facebook_ads,10844491.0,389.0
google_ads,11089575.0,390.5
instagram_ads,11096726.0,393.5
referal,10491527.0,395.5
vk_ads,11334978.0,393.0
yandex-direct,10969061.5,392.5
